# Deploying Iris-detection model using Vertex AI


## Overview

In this tutorial, you build a scikit-learn model and deploy it on Vertex AI using the custom container method. You use the FastAPI Python web server framework to create a prediction endpoint. You also incorporate a preprocessor from training pipeline into your online serving application.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training) and [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).

### Objective

In this notebook, you learn how to create, deploy and serve a custom classification model on Vertex AI. This notebook focuses more on deploying the model than on the design of the model itself.


This tutorial uses the following Vertex AI services and resources:

- Vertex AI models
- Vertex AI endpoints

The steps performed include:

- Train a model that uses flower's measurements as input to predict the class of iris.
- Save the model and its serialized pre-processor.
- Build a FastAPI server to handle predictions and health checks.
- Build a custom container with model artifacts.
- Upload and deploy custom container to Vertex AI Endpoints.

### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small and popular dataset for machine learning experiments. Each instance has four numerical features, which are different measurements of a flower, and a target label that
categorizes the flower into: **Iris setosa**, **Iris versicolour** and **Iris virginica**.

This tutorial uses [a version of the Iris dataset available in the
scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry
* Cloud Build

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing) and [Cloud Build pricing](https://cloud.google.com/build/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages



In [1]:
# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform

### Set Google Cloud project information
Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "loyal-glass-474108-f0"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [3]:
BUCKET_URI = f"gs://mlops-course-22f3003058"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [4]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-course-22f3003058/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-course-22f3003058' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

In [5]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import the required libraries

In [6]:
import os
import sys
import datetime

### Configure resource names

Set a name for the following parameters:

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that is pushed to the repository.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI model resource.

In [7]:
TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
MODEL_ARTIFACT_DIR = f"my-models/iris-classifier-week-1/{TIMESTAMP}"  # @param {type:"string"}
REPOSITORY = "iris-classifier-repo"  # @param {type:"string"}
IMAGE = "iris-classifier-img"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "iris-classifier"  # @param {type:"string"}

# Set the defaults if no names were specified
if MODEL_ARTIFACT_DIR == "[your-artifact-directory]":
    MODEL_ARTIFACT_DIR = "custom-container-prediction-model"

if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-container-prediction"

if IMAGE == "[your-image-name]":
    IMAGE = "sklearn-fastapi-server"

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "sklearn-custom-container"

## Simple Decision Tree model
Build a Decision Tree model on iris data

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

data_1 = pd.read_csv('data/data_v1.csv')
data_1.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.8,4.0,1.2,0.2,setosa
1,5.7,4.4,1.5,0.4,setosa
2,5.4,3.9,1.3,0.4,setosa
3,5.1,3.5,1.4,0.3,setosa
4,5.7,3.8,1.7,0.3,setosa


In [9]:
train, test = train_test_split(data_1, test_size = 0.4, stratify = data_1['species'], random_state = 42)
X_train_1 = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train_1 = train.species
X_test_1 = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test_1 = test.species

In [10]:
mod_dt = DecisionTreeClassifier(max_depth = 3, random_state = 1)
mod_dt.fit(X_train_1,y_train_1)
prediction=mod_dt.predict(X_test_1)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_1)))

The accuracy of the Decision Tree is 0.951


In [11]:
import pickle
import joblib
os.makedirs("artifacts", exist_ok=True)
joblib.dump(mod_dt, "artifacts/model_1.joblib")

['artifacts/model_1.joblib']

In [12]:
data_2 = pd.read_csv('data/data_v1.csv')
data_2.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.8,4.0,1.2,0.2,setosa
1,5.7,4.4,1.5,0.4,setosa
2,5.4,3.9,1.3,0.4,setosa
3,5.1,3.5,1.4,0.3,setosa
4,5.7,3.8,1.7,0.3,setosa


In [13]:
train, test = train_test_split(data_1, test_size = 0.4, stratify = data_2['species'], random_state = 42)
X_train_2 = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train_2 = train.species
X_test_2 = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test_2 = test.species

In [14]:
mod_dt_2 = DecisionTreeClassifier(max_depth = 3, random_state = 1)
mod_dt_2.fit(X_train_2,y_train_2)
prediction=mod_dt_2.predict(X_test_2)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_2)))

The accuracy of the Decision Tree is 0.951


In [15]:
os.makedirs("artifacts", exist_ok=True)
joblib.dump(mod_dt, "artifacts/model_2.joblib")

['artifacts/model_2.joblib']

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.joblib` (model artifact)
* `preprocessor.pkl` (model artifact)

Run the following commands to upload your files:

In [16]:
!gsutil cp artifacts/model_1.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://artifacts/model_1.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      


In [17]:
!gsutil cp artifacts/model_2.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://artifacts/model_2.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
